In [1]:
from datetime import datetime, timedelta
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline
%config IPCompleter.greedy = True
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
import cbpro
import os
from pathlib import Path
import seaborn as sns
# sns.set_context("notebook", font_scale=1.)
# sns.set_style("whitegrid")

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
import tensorflow_probability as tfp

tf.distribute.OneDeviceStrategy(device="/gpu:0")
# policy = tf.keras.mixed_precision.Policy("mixed_float16")
# tf.keras.mixed_precision.experimental.set_policy(policy)
np.set_printoptions(suppress=True)

public_client = cbpro.PublicClient()


In [2]:
class MinerMeta(type):
    # def __init__(self):
    #     result = getattr(self, "df", None)
    #     if result is None:
    #         self.df = self.compile_historic(read_csv=True)

    def compile_historic(self, num_days=100, write_csv=False, read_csv=False):
        file = Path.cwd() / f"{self.coin}_histdata.csv"
        if read_csv is True:
            df = pd.read_csv(file, index_col="time", infer_datetime_format=True)
            return df
        else:
            finish = datetime.now()
            start = finish - timedelta(num_days)
            delta = timedelta(hours=300)
            df = pd.DataFrame()

            while finish > start:
                historic = public_client.get_product_historic_rates(
                    f"{self.coin}-USD",
                    granularity=3600,
                    start=start,
                    end=start + delta,
                )
                start += delta
                df = df.append(historic, ignore_index=True, verify_integrity=True)
            df.columns = ["time", "low", "high", "open", "close", "volume"]
            date_time = pd.to_datetime(df["time"], unit="s")
            # df.set_index("time", inplace=True)
            df.sort_values(by="time", ascending=False, inplace=True)

            timestamp_s = date_time.map(pd.Timestamp.timestamp)
            day = 24 * 60 * 60
            year = (365.2425) * day
            df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
            df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
            df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
            df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
            df.reset_index(drop=True, inplace=True)
            df["time"] = pd.to_datetime(df["time"], unit="s")
            df.set_index("time", inplace=True, verify_integrity=False)
            df.sort_index(ascending=False)
            if write_csv is True:
                df.to_csv(file, index=True)

            return df

    def get_day_stats(self):
        result = getattr(self, "day_stats", None)
        if result is None:
            ticker = public_client.get_product_24hr_stats(f"{self.coin}-USD")
            df = pd.DataFrame.from_dict(ticker, orient="index")
            self.day_stats = df
        return df

    def year_day_fft(self, col):
        df = self.compile_historic()
        today = datetime.today()
        ylim = int(df[col].max())
        plt.figure(figsize=(15, 10))

        fft = tf.signal.rfft(df[col])
        f_per_dataset = np.arange(0, len(fft))

        n_samples_h = 1
        hours_per_year = 24 * 365.2524
        hours_per_week = 24 * 7
        years_per_dataset = n_samples_h / (hours_per_year)
        hours_per_dataset = n_samples_h / (hours_per_week)

        f_per_year = f_per_dataset / years_per_dataset
        f_per_week = f_per_dataset / hours_per_dataset
        plt.step(f_per_week, np.abs(fft))
        plt.xscale("log")
        # plt.ylim(1000, ylim)
        # plt.xlim([0.1, max(plt.xlim())])
        plt.xticks([1, 7], labels=["1/Week", "1/day"])
        plt.xlabel("Frequency (log scale)")

        return plt.show()

    def ttsplit_norm(self, df, split_time=0.7, feature_plot=False):
        # train_df Test Split
        n = len(df)
        train_df = df[0 : int(n * 0.7)]
        val_df = df[int(n * 0.7) : int(n * 0.9)]
        test_df = df[int(n * 0.9) :]
        # Normalize the Data
        train_df_mean = train_df.mean()
        train_df_std = train_df.std()

        train_df = (train_df - train_df_mean) / train_df_std
        val_df = (val_df - train_df_mean) / train_df_std
        test_df = (test_df - train_df_mean) / train_df_std

        # Create Feature Plot if wanted
        if feature_plot is True:
            df_std = (df - train_df_mean) / train_df_std
            df_std = df_std.melt(var_name="Column", value_name="Normalized")
            plt.figure(figsize=(12, 6))
            ax = sns.violinplot(x="Column", y="Normalized", data=df_std)
            ax.set_xticklabels(df.keys(), rotation=90)
            ax.set_title("train_dfing Data Feature Dist with whole DF Mean")

            return train_df, val_df, test_df

        return train_df, val_df, test_df

    def __call__(self, *args, **kwargs):

        cls = type.__call__(self, *args)

        # setattr(cls, "compile_historic", self.compile_historic)
        # setattr(cls, "year_day_fft", self.year_day_fft)
        # setattr(cls, "ttsplit_norm", self.ttsplit_norm)
        # setattr(cls, "get_day_stats", self.get_day_stats)
        # setattr(cls, "day_stats", self.get_day_stats())

        # for key, value in historic.items():
        #     setattr(cls, "hist_" + key, value)
        # for key, value in ticker.items():
        #     setattr(cls, "tick_" + key, value)

        return cls


In [5]:
class eth(metaclass=MinerMeta):
    coin = "eth"


df = eth.compile_historic(read_csv=True)


In [6]:
def plot_forecast(
    x,
    y,
    forecast_mean,
    forecast_scale,
    forecast_samples,
    title=None,
    x_locator=None,
    x_formatter=None,
):
    """Plot a forecast distribution against the 'true' time series."""
    colors = sns.color_palette()
    c1, c2 = colors[0], colors[1]
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1)

    num_steps = len(y)
    num_steps_forecast = forecast_mean.shape[-1]
    num_steps_train = num_steps - num_steps_forecast

    ax.plot(x, y, lw=2, color=c1, label="ground truth")

    forecast_steps = np.arange(
        x[num_steps_train],
        int(x[num_steps_train]) + int(num_steps_forecast),
        dtype=x.dtype,
    )

    ax.plot(forecast_steps, forecast_samples.T, lw=1, color=c2, alpha=0.1)

    ax.plot(forecast_steps, forecast_mean, lw=2, ls="--", color=c2, label="forecast")
    ax.fill_between(
        forecast_steps,
        forecast_mean - 2 * forecast_scale,
        forecast_mean + 2 * forecast_scale,
        color=c2,
        alpha=0.2,
    )

    ymin, ymax = min(np.min(forecast_samples), np.min(y)), max(
        np.max(forecast_samples), np.max(y)
    )
    yrange = ymax - ymin
    ax.set_ylim([ymin - yrange * 0.1, ymax + yrange * 0.1])
    ax.set_title("{}".format(title))
    ax.legend()

    if x_locator is not None:
        ax.xaxis.set_major_locator(x_locator)
        ax.xaxis.set_major_formatter(x_formatter)
        fig.autofmt_xdate()

    return fig, ax


In [ ]:
time_series = df["close"]


def build_linear_model(time_series):
    trend = tfp.sts.LocalLinearTrend(observed_time_series=time_series)
    day_of_week = tfp.sts.Seasonal(
        num_seasons=7,
        num_steps_per_season=24,
        observed_time_series=time_series,
        name="day_of_week",
    )
    time_of_day = tfp.sts.Seasonal(
        num_seasons=24,
        num_steps_per_season=1,
        observed_time_series=time_series,
        name="time_of_day",
    )
    model = tfp.sts.Sum(
        [trend, day_of_week, time_of_day], observed_time_series=time_series
    )
    return model


model = build_linear_model(time_series)
var_posteriors = tfp.sts.build_factored_surrogate_posterior(model=model)


# Minimize Variational loss

In [ ]:
num_var_steps = 200

elbo_loss_curve = tfp.vi.fit_surrogate_posterior(
    target_log_prob_fn=model.joint_log_prob(observed_time_series=time_series),
    surrogate_posterior=var_posteriors,
    optimizer=tf.optimizers.Adam(lr=0.1),
    num_steps=num_var_steps,
    jit_compile=True,
)

plt.plot(elbo_loss_curve)
plt.show()


In [ ]:
q_samples = var_posteriors.sample(50)

for param in model.parameters:
    print(
        "{}: {} +- {}".format(
            param.name,
            np.mean(q_samples[param.name], axis=0),
            np.std(q_samples[param.name], axis=0),
        )
    )


In [ ]:
forecast = tfp.sts.forecast(
    model,
    observed_time_series=time_series,
    parameter_samples=q_samples,
    num_steps_forecast=24,
)


In [ ]:
def plot_forecast(
    x,
    y,
    forecast_mean,
    forecast_scale,
    forecast_samples,
    title=None,
    x_locator=None,
    x_formatter=None,
):
    """Plot a forecast distribution against the 'true' time series."""
    colors = sns.color_palette()
    c1, c2 = colors[0], colors[1]
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1)

    num_steps = len(y)
    num_steps_forecast = forecast_mean.shape[-1]
    num_steps_train = num_steps - num_steps_forecast

    ax.plot(x, y, lw=2, color=c1, label="ground truth")

    forecast_steps = np.arange(
        x[num_steps_train], int(x[num_steps_train]) + int(num_steps_forecast), dtype=x.dtype
    )

    ax.plot(forecast_steps, forecast_samples.T, lw=1, color=c2, alpha=0.1)

    ax.plot(forecast_steps, forecast_mean, lw=2, ls="--", color=c2, label="forecast")
    ax.fill_between(
        forecast_steps,
        forecast_mean - 2 * forecast_scale,
        forecast_mean + 2 * forecast_scale,
        color=c2,
        alpha=0.2,
    )

    ymin, ymax = min(np.min(forecast_samples), np.min(y)), max(
        np.max(forecast_samples), np.max(y)
    )
    yrange = ymax - ymin
    ax.set_ylim([ymin - yrange * 0.1, ymax + yrange * 0.1])
    ax.set_title("{}".format(title))
    ax.legend()

    if x_locator is not None:
        ax.xaxis.set_major_locator(x_locator)
        ax.xaxis.set_major_formatter(x_formatter)
        fig.autofmt_xdate()

    return fig, ax


In [ ]:
num_samples = int(10)

forecast_mean, forecast_scale, forecast_samples = (
    forecast.mean().numpy()[..., 0],
    forecast.stddev().numpy()[..., 0],
    forecast.sample(num_samples).numpy()[..., 0],
)


In [ ]:
import matplotlib.dates as mdates

x_loc = mdates.YearLocator(3)
fmt = mdates.DateFormatter("%Y")

fig, ax = plot_forecast(
    time_series.index,
    time_series.values,
    forecast_mean,
    forecast_scale,
    forecast_samples,
    # title="Eth Close Forecast",
    # x_locator=x_loc,
    # x_formatter=fmt,
)
# ax.axvline(dates[-num_forecast_steps], linestyle="--")
fig.autofmt_xdate()


In [ ]:
new_data = eth.compile_historic(num_days=1)
dff = eth.compile_historic(num_days=10)

